In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Customer Container for prediction

### Configuration

In [1]:
! pip install --upgrade --quiet  google-cloud-aiplatform \
                                 google-cloud-storage \
                                 uvicorn[standard] fastapi

In [2]:
PROJECT_ID = "ai-hangsik" 
LOCATION = "us-central1" 
BUCKET_URI = f"gs://sllm_0116" 

In [3]:
import os
import sys
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [5]:
MODEL_ARTIFACT_DIR = "custom-container-prediction-model"
REPOSITORY = "custom-container-prediction"
IMAGE = "sklearn-fastapi-server"
MODEL_DISPLAY_NAME = "sklearn-custom-container"

### Build customer container

In [9]:
!pwd
!ls -al

/home/jupyter/llmOps_vertexAI/custom_container
total 96
drwxr-xr-x 4 jupyter jupyter  4096 Feb  3 08:46 .
drwxr-xr-x 4 jupyter jupyter  4096 Feb  3 08:04 ..
drwxr-xr-x 2 jupyter jupyter  4096 Feb  3 08:12 .ipynb_checkpoints
-rw-r--r-- 1 jupyter jupyter   143 Feb  3 07:44 Dockerfile
drwxr-xr-x 2 jupyter jupyter  4096 Feb  3 07:40 app
-rw-r--r-- 1 jupyter jupyter 31350 Feb  3 08:46 custom_container.ipynb
-rw-r--r-- 1 jupyter jupyter    88 Feb  3 07:40 instances.json
-rw-r--r-- 1 jupyter jupyter 34388 Feb  3 08:25 model_build.ipynb
-rw-r--r-- 1 jupyter jupyter    40 Feb  3 07:43 requirements.txt


In [ ]:
%mkdir app

In [10]:
# Copy model to GCS
!gsutil cp app/model.pickle {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://app/model.pickle [Content-Type=application/octet-stream]...
/ [1 files][  2.6 KiB/  2.6 KiB]                                                
Operation completed over 1 objects/2.6 KiB.                                      


In [11]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import joblib
import json
import numpy as np
import pickle
import os

from google.cloud import storage
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris

app = FastAPI()
gcs_client = storage.Client()


# Download model file from GCS
with open("model.pickle", 'wb') as model_f:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/model.pickle", model_f
    )

# Load model file stored in local was downloaded from GCS
with open("model.pickle", "rb") as f:
    model = pickle.load(f)

_class_names = load_iris().target_names
_model = model

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {"status":"OK"}

@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    inputs = np.asarray(instances)
    outputs = _model.predict(inputs)

    return {"predictions": [_class_names[class_num] for class_num in outputs]}


Overwriting app/main.py


In [12]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Overwriting app/prestart.sh


In [13]:
%%writefile instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

Overwriting instances.json


In [14]:
%%writefile requirements.txt
numpy
scikit-learn
google-cloud-storage

Overwriting requirements.txt


In [15]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

Overwriting Dockerfile


In [16]:
! docker build --tag="{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}" .

Sending build context to Docker daemon  174.6kB
Step 1/4 : FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9
 ---> 95735a0480b5
Step 2/4 : COPY ./app /app
 ---> 4eee73b2c4a5
Step 3/4 : COPY requirements.txt requirements.txt
 ---> 5dc17ccadfc6
Step 4/4 : RUN pip install -r requirements.txt
 ---> Running in 93d7e3e5e539
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
!docker stop local-iris

local-iris


In [19]:
!docker rm local-iris

local-iris


In [20]:
! docker ps -a

CONTAINER ID   IMAGE                                                                           COMMAND                  CREATED      STATUS                  PORTS     NAMES
d7598bff538e   us-central1-docker.pkg.dev/ai-hangsik/custom-inference-gpu/tgi-release:latest   "./entrypoint.sh '--…"   2 days ago   Exited (0) 2 days ago             stupefied_williams


In [21]:
! docker images

REPOSITORY                                                                                                                TAG         IMAGE ID       CREATED             SIZE
us-central1-docker.pkg.dev/ai-hangsik/custom-container-prediction/sklearn-fastapi-server                                  latest      3270dfb5d66f   26 seconds ago      1.4GB
<none>                                                                                                                    <none>      afbfba19495d   About an hour ago   1.4GB
<none>                                                                                                                    <none>      e5aa46fa9e71   2 hours ago         1.75GB
<none>                                                                                                                    <none>      0b57ae845577   2 hours ago         1.75GB
<none>                                                                                                                    <n

In [22]:
! docker run -d -p 80:8080 \
        --name=local-iris \
        -e AIP_HTTP_PORT=8080 \
        -e AIP_HEALTH_ROUTE=/health \
        -e AIP_PREDICT_ROUTE=/predict \
        -e AIP_STORAGE_URI={BUCKET_URI}/{MODEL_ARTIFACT_DIR} \
        "{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"

df222cfa7f551d9f5ef97316b11f17fbabe5f722427f027986bc917ecef5382f


In [24]:
! docker ps -a

CONTAINER ID   IMAGE                                                                                      COMMAND                  CREATED          STATUS                  PORTS                                   NAMES
df222cfa7f55   us-central1-docker.pkg.dev/ai-hangsik/custom-container-prediction/sklearn-fastapi-server   "/start.sh"              12 seconds ago   Up 12 seconds           0.0.0.0:80->8080/tcp, :::80->8080/tcp   local-iris
d7598bff538e   us-central1-docker.pkg.dev/ai-hangsik/custom-inference-gpu/tgi-release:latest              "./entrypoint.sh '--…"   2 days ago       Exited (0) 2 days ago                                           stupefied_williams


In [26]:
!docker logs df222cfa7f55

Checking for script in /app/prestart.sh
Running script /app/prestart.sh
{"loglevel": "info", "workers": 24, "bind": "0.0.0.0:8080", "graceful_timeout": 120, "timeout": 120, "keepalive": 5, "errorlog": "-", "accesslog": "-", "workers_per_core": 1.0, "use_max_workers": null, "host": "0.0.0.0", "port": "8080"}
[2025-02-03 08:56:08 +0000] [1] [INFO] Starting gunicorn 23.0.0
[2025-02-03 08:56:08 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2025-02-03 08:56:08 +0000] [1] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2025-02-03 08:56:08 +0000] [7] [INFO] Booting worker with pid: 7
[2025-02-03 08:56:08 +0000] [8] [INFO] Booting worker with pid: 8
[2025-02-03 08:56:08 +0000] [9] [INFO] Booting worker with pid: 9
[2025-02-03 08:56:08 +0000] [10] [INFO] Booting worker with pid: 10
[2025-02-03 08:56:08 +0000] [11] [INFO] Booting worker with pid: 11
[2025-02-03 08:56:08 +0000] [12] [INFO] Booting worker with pid: 12
[2025-02-03 08:56:08 +0000] [13] [INFO] Booting worker with pid: 

In [27]:
! curl localhost/health

{"status":"OK"}

In [28]:
! curl -X POST \
  -d @instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  localhost/predict

{"predictions":["versicolor","setosa"]}

In [29]:
!docker stop local-iris

local-iris


In [30]:
!docker rm local-iris

local-iris


In [31]:
!docker images

REPOSITORY                                                                                                                TAG         IMAGE ID       CREATED             SIZE
us-central1-docker.pkg.dev/ai-hangsik/custom-container-prediction/sklearn-fastapi-server                                  latest      3270dfb5d66f   4 minutes ago       1.4GB
<none>                                                                                                                    <none>      afbfba19495d   About an hour ago   1.4GB
<none>                                                                                                                    <none>      e5aa46fa9e71   2 hours ago         1.75GB
<none>                                                                                                                    <none>      0b57ae845577   3 hours ago         1.75GB
<none>                                                                                                                    <n